## Create Service Principal

This code calls the Graph API to create a service principal.  This is a multi step process.

1. Create the application and authentication secret.
1. Grab the secret and appId from the API response.
1. Store the secret and appId for later usage in KeyVault.
1. Call the create Service Principal API with the appId to convert the new app to a service principal.

Resources:

1. Create Application - https://learn.microsoft.com/en-us/graph/api/application-post-applications?view=graph-rest-1.0&tabs=http ((Note, the identity you call this API from must have ApplicationReadWrite.All permissions)
1. Create Service Principal - https://learn.microsoft.com/en-us/graph/api/serviceprincipal-post-serviceprincipals?view=graph-rest-1.0&tabs=http 

In [ ]:
pip install requests msal

In [ ]:
tenantName = "Tenant1"
keyVault = "cgmmlservicevault"

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login.
#Note: for most of the other notebooks we're not passing in a scope, this is because the code in aadservice.property
#defaults the scope to a scope for the Power BI API.   Since we're calling the azure management API we need a different
#scope.
from services.aadservice import AadService
scope = 'https://graph.microsoft.com/.default'
cred = AadService.get_credential()

#with the credential object, get the token for the azure management scope.
aadToken = cred.get_token(scope).token
headers =  {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + aadToken}

In [ ]:
import requests
import json

apiUrl = f'https://graph.microsoft.com/v1.0/applications'       

body = {
    "displayName": tenantName,
    "passwordCredentials":[
        {
            "displayName": "auth secret"
        }
    ]
}

apiResponse = requests.post(apiUrl, headers=headers, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))

In [ ]:
from services.secretservice import SecretService

#Grab the app id and client secret for the created application.
appId = apiResponse["appId"]
credentials = apiResponse["passwordCredentials"]
credential = credentials[0]
secret = credential["secretText"]

#Here we're mocking up a tenantName to store the secrets under that corresponds to the name of the app we created.
appIdKey = f'{tenantName}Id'
appSecretKey = f'{tenantName}Secret'

SecretService.store_secret_byname(keyVault, appIdKey, appId)
SecretService.store_secret_byname(keyVault, appSecretKey, secret)

In [ ]:
import requests
import json

#appId = str(uuid.uuid4())
apiUrl = f'https://graph.microsoft.com/v1.0/servicePrincipals'       

body = {
  "appId": appId
}

apiResponse = requests.post(apiUrl, headers=headers, data=json.dumps(body))

#error handling for create capacity
if apiResponse.status_code != 201 and apiResponse.status_code != 200:
    description = f'Error creating capacity:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
    print(description)
else:
    apiResponse = json.loads(apiResponse.text)
    print(json.dumps(apiResponse,indent=2))